## Welcome to the Python Neuroimaging Workshop!

- Welcome statement 
- What we'll cover
- Goals of the workshop
- 

Welcome to our Python Neuroimaging Analysis workshop. In this workshop we'll teach you the tools to perform reproducible analyses on functional neuroimaging (fMRI) data using Python and Jupyter notebooks. The focus of this workshop is to learn the basics of image manipulation and analysis using *already pre-processed data*; data pre-processing is a whole other topic that we'll briefly touch on. We'll start with a basic overview of how we can load, view, and play around with MR images using Python's neuroimaging libraries (<code>Nilearn</code>), then we'll eventually perform a full functional connectivity analysis in order to make inferences about whole-brain activity. 

### Why Python?

Python is rapidly becoming the standard language for data analysis, visualization and automated workflow building. It is a free and open-source software that is relatively easy to pick up by new programmers. In addition with Python packages such as <code>Jupyter</code> one can keep an interactive code journal of analysis - this is what we'll be using in the workshop, and is, in fact, what you are currently using to view this workshop! Using Jupyter notebooks allows you to keep a record of all the steps in your analysis enabling transparency in analysis and ease of code sharing. 

Another advantage of Python is that it is maintained by a large user-base. Anyone can easily make their own Python packages for others to use, therefore there exists a *very* large codebase for you to take advantage of for your neuroimaging analysis; from basic statistical analysis to advanced machine learning and multivariate methods!

We'll begin with a basic overview of MR data then move into how we can work with MR data in python:

### Types of MR Scans

<img src="../static/images/mr_scan_types.png" alt="Drawing" align="middle" width="500px">

For this tutorial, we'll be focusing on T1w and resting state fMRI scans.

### Neuroimaging File Formats

|Format Name | File Extension | Origin |
|---|---|---|
| Analyze | .img/.hdr | Analyze Software, Mayo Clinic |
| DICOM | none | ACR/NEMA Consortium |
| NIfTI | .nii or .img/.hdr | Neuroimaging Informatics Technology Initiative |
| MINC | .mnc | Montreal Neurological Institute |
| NRRD | .nrrd | |

<img src="../static/images/dicom_to_nifti.png" alt="Drawing" align="middle" width="300px"/>

From the MRI scanner, images are initially collected in the DICOM format and can be converted to NIfTI using [dcm2niix](https://github.com/rordenlab/dcm2niix).

### Intro to NIfTI

NIfTI is one of the most ubiquitous file formats for storing neuroimaging data. We'll cover a few details to get started working with them. If you're interested in learning more about NIfTI images, we highly recommend [this blog post about the NIfTI format](http://brainder.org/2012/09/23/the-nifti-file-format/).

***

### Working with NIfTI packages in Python (Nilearn)

[Nilearn](https://nilearn.github.io/) is a functional neuroimaging analysis and visualization library that wraps up a whole bunch of high-level operations (machine learning, statistical analysis, data cleaning, etc…) in easy-to-use commands.

### Setting up
The first thing we’ll do is to important some Python modules that will allow us to use Nilearn:

In [2]:
import os
import matplotlib.pyplot as plt
from nilearn import image as img
from nilearn import plotting as plot
%matplotlib inline #for inline visualization in jupyter notebook

/home/jerry/.pyenv/versions/3.6.0/envs/scwg_neuroimaging/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)
UsageError: unrecognized arguments: #for inline visualization in jupyter notebook


Notice that we imported two things:

1. <code> image as img </code> - allows us to load NIfTI images
2. <code> plotting as plot </code> - allows us to use Nilearn's plotting library for easy visualization

Let's grab an example scan to work with:

In [5]:
t1_img = img.load_img('../data/ds000030/')

ValueError: File not found: '../data/ds000030/sub-10171/anat/sub-10171_T1w.nii.gz'

Loading in a NIfTI file with <code>nilearn</code> gives us a special type of data object which encodes all the information available in the NIfTI file. To see all of these attributes, type `t1_img.` and <kbd>Tab</kbd>.  
There are three main attributes that we'll discuss today:

#### 1. [Header](http://nipy.org/nibabel/nibabel_images.html#the-image-header): contains metadata about the image, such as image dimensions, data type, etc.

In [ ]:
t1_hdr = t1_img.header
print(t1_hdr)

The <code>t1_img</code> image header is a [Python dictionary](https://www.w3schools.com/python/python_dictionaries.asp). Dictionaries are containers which can store a variety of information called **values** which can be accessed using a **key**. 

Similar to <code>t1_img</code> in which attributes can be accessed via typing <code>t1_img.</code> and hitting <kbd>Tab</kbd>, you can do the same with <code>t1_hdr</code>. In particular, we'll be using a *method* (a process) belonging to <code>t1_hdr</code> that will allow you to view the keys associated with it:

In [ ]:
t1_hdr.keys()

Notice that *methods* include <code>()</code> at end whereas *attributes* do not. 

The key difference between a method and an attribute is:
- Attributes are *stored values* kept within an object
- Methods are *processes* that we can run using the object. Usually a method takes *attributes*, performs an operation on them, then returns it for you to use.

You can view the *methods* and *attributes* of a particular object by typing:

~~~python
?t1_img
~~~

The output above is a list of **keys** you can use from <code>t1_hdr</code> to access **values**. 

We can access the value stored by a given key by typing:

```python
t1_hdr['<key_name>']
```

<b>EXERCISE:</b> Extract the value of <code>pixdim</code> from <code>t1_hdr</code>

In [ ]:
t1_hdr['pixdim']

#### MR Data
As you've seen above, the header contains useful information that gives us information about the properties (metadata) associated with the MR data we've loaded in. Now we'll move in to loading the actual *image data itself*. We can achieve this by using the *method* called <code>t1_img.get_data()</code> 

In [ ]:
data = t1_img.get_data()
print(data)

<code>t1_img.get_data()</code> returns an <code>array</code> object which contains 3 dimensions. You can think of <code>data</code> as a 3D version of a picture (more accurately, a volume):


<img src="../static/images/numpy_arrays.png" alt="Drawing" align="middle" width="500px"/>

A 3D array contains a value for each (x,y,z) coordinate in our image. You can think of our <code>data</code> object as this 3D volume made up of cubes, and these cubes make up a "picture" of the MR scan as shown below:



<img src="http://www.sprawls.org/mripmt/MRI10/MR10-2.jpg" alt="Drawing" align="middle" width="500px"/>

Now let's inspect some *attributes* of this <code>data</code> object!

First, let's get an idea about how big it is. The <code>.shape</code> attribute tells us this information:

In [ ]:
data.shape

The 3 numbers given here represent the number of values *along a respective dimension (x,y,z)*. That means in total there are:

$$x * y * z = value$$ voxels in total!